In [1]:
# installation: 
! pip install pandas
! pip install datasets
! pip install transformers
! pip install torch

In [2]:
import pandas as pd
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

c:\users\micha\appdata\local\programs\python\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
lyrics_df = pd.read_csv('./datasets/df_songs.csv')
lyrics_df.head()

Artist                               Song  \
0         Jerry Lee Lewis                    Goodnight Irene   
1             Buddy Holly                Raining in My Heart   
2           Elvis Presley                          Blue Moon   
3  The Velvet Underground  Ride Into the Sun Session Outtake   
4  The Velvet Underground                              Jesus   

                                              Lyrics  LyricsWordCount  
0  Last Saturday night I got married Me and my wi...               80  
1  The sun is out The sky is blue Theres not a cl...               80  
2  Blue moon You saw me standing alone Without a ...               81  
3  Ride Into the Sun Session Outtake Lyrics Looki...               81  
4  Jesus help me find my proper place Jesus help ...               81

In [4]:
test_set = lyrics_df.sample(n=400)
lyrics_df = lyrics_df.loc[~lyrics_df.index.isin(test_set.index)]

In [5]:
lyrics_dataset = Dataset.from_pandas(lyrics_df)
lyrics_dataset = lyrics_dataset.remove_columns(['Artist', 'Song', 'LyricsWordCount'])

In [6]:
data_split = lyrics_dataset.train_test_split(test_size=0.10, shuffle=True)

lyris_dataset_train = data_split["train"]
lyris_dataset_valid = data_split["test"]

In [14]:
#TODO: Create suitable test datset, keep only first 20 words
test_set['True_end_lyrics'] = test_set['Lyrics'].str.split().str[-20:].apply(' '.join)
test_set['Lyrics'] = test_set['Lyrics'].str.split().str[:-20].apply(' '.join)

---

In [8]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [9]:
# NCIHT SICHER OB DAS GOOD PRACTICE IST
tokenizer.pad_token = tokenizer.eos_token

In [10]:
dataset_train = lyris_dataset_train.map(lambda examples: tokenizer(examples["Lyrics"], truncation=True))
dataset_valid = lyris_dataset_valid.map(lambda examples: tokenizer(examples["Lyrics"], truncation=True))

100%|███████████████████████████████████████████████████████████████████████████████| 523/523 [00:01<00:00, 383.45ex/s]


In [15]:
trainings_args = TrainingArguments(
    output_dir="gpt2_train_output",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    learning_rate=1e-5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=trainings_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_valid,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

In [16]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: Lyrics, __index_level_0__. If Lyrics, __index_level_0__ are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
c:\users\micha\appdata\local\programs\python\python38\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4699
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 740
  Number of trainable parameters = 124439808


KeyboardInterrupt: 